In [ ]:
! pip install chart-studio
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
import string
import numpy as np
import time
import os




In [3]:
file_path =('/content/kor.txt')
lines = open(file_path, encoding='UTF-8').read().strip().split('\n')
lines[5000:5010]

print("total number of record:", len(lines))

total number of record: 5870


In [14]:
exclude = set(string.punctuation) # Set of all special characters
remove_digits = str.maketrans('', '', string.digits) # Set of all


In [15]:
def preprocess_kor_sentence(sentence):
        #    '''Function to preprocess English sentence'''
       sent = sent.lower() # lower casing
       sent = re.sub("'", '', sent) # remove the quotation marks if any
       sent = ''.join(ch for ch in sent if ch not in exclude)
       sent = sent.translate(remove_digits) # remove the digits
       sent = sent.strip()
       sent = re.sub(" +", " ", sent) # remove extra spaces
       sent = '<start> ' + sent + ' <end>' # add <start> and <end> tokens
       return sent

def preprocess_eng_sentence(sentence):
         sent = re.sub("'", '', sent) # remove the quotation marks if any
         sent = ''.join(ch for ch in sent if ch not in exclude)
         sent = sent.strip()
         sent = re.sub(" +", " ", sent) # remove extra spaces
         sent = '<start> ' + sent + ' <end>' # add <start> and <end> tokens
         return sent

In [16]:
def preprocess_kor_sentence(sentence):
    # Define 'sent' and do some processing
    sent = sentence
    # Other preprocessing steps using 'sent'
    return sent

def preprocess_eng_sentence(sentence):
    # Define 'sent' and do some processing
    sent = sentence
    # Other preprocessing steps using 'sent'
    return sent

sent_pairs = []
for line in lines:
    sent_pair = []
    kor = line.rstrip().split('\t')[0]
    eng = line.rstrip().split('\t')[1]
    kor = preprocess_kor_sentence(kor)
    sent_pair.append(kor)
    eng = preprocess_eng_sentence(eng)
    sent_pair.append(eng)
    sent_pairs.append(sent_pair)
sent_pairs[5000:5010]

[['My daughter is not yet three years old.', '내 딸은 아직 3 살이 안 됐어.'],
 ['My family goes to Australia every year.', '우리 가족은 매년 오스트레일리아에 간다.'],
 ['Nothing is certain but death and taxes.', '죽음과 세금만큼 확실한 것은 없어.'],
 ['Onions cook more quickly than potatoes.', '양파는 감자보다 빨리 익는다.'],
 ["Our family doesn't celebrate Christmas.", '우리 가족은 크리스마스를 축하하지 않아.'],
 ['That was the only way we could do that.', '그게 우리가 할 수 있는 유일한 방법이었습니다.'],
 ['The earth is much larger than the moon.', '지구는 달보다 훨씬 더 넓어.'],
 ['The plane took off from Boston at 2:30.', '비행기는 오후 2시 30분에 보스턴을 떠났다.'],
 ['The rebels took control of the capital.', '반역자들이 수도를 점령했다.'],
 ['The rebels took control of the capital.', '반역자들이 수도를 접수했다.']]

In [17]:
class LanguageIndex:
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()

    def create_index(self, language):
        for phrase in language:
            self.vocab.update(phrase.split(' '))

        self.vocab = sorted(self.vocab)

        self.word2idx['<pad>'] = 0
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1

        for word, index in self.word2idx.items():
            self.idx2word[index] = word



In [32]:
# Tokenization and Padding
def load_dataset(pairs, num_examples):

    # Index language using the class defined above
    inp_lang = LanguageIndex()
    inp_lang.create_index(en for en, ma in pairs)

    targ_lang = LanguageIndex()
    targ_lang.create_index(ma for en, ma in pairs)

    # Vectorize the input and target languages
    input_tensor = [[inp_lang.word2idx[s] for s in en.split(' ')] for en, ma in pairs]
    target_tensor = [[targ_lang.word2idx[s] for s in ma.split(' ')] for en, ma in pairs]

    # Calculate max_length of input and output tensor
    max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)

    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, maxlen=max_length_inp, padding='post')
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, maxlen=max_length_tar, padding='post')

    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar


In [33]:
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(sent_pairs, len(lines))

TypeError: ignored

In [34]:
## Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1, random_state = 101)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

NameError: ignored

In [40]:
# Model parameters
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 32
N_BATCH = BUFFER_SIZE // BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [41]:
def lstm(units):

    return tf.keras.layers.LSTM(units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_activation='sigmoid',
                                   recurrent_initializer='glorot_uniform')

In [48]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(self.enc_units, return_sequences=True, return_state=True)

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state_h, state_c = self.lstm(x, initial_state=hidden)
        return output, [state_h, state_c]

    def initialize_hidden_state(self):
        return [tf.zeros((self.batch_sz, self.enc_units)), tf.zeros((self.batch_sz, self.enc_units))]

class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(self.dec_units, return_sequences=True, return_state=True)
        self.fc = tf.keras.layers.Dense(vocab_size)

        # Used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, x, hidden, enc_output):
        hidden_with_time_axis = tf.expand_dims(hidden[0], 1)
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))

        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)

        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        output, state_h, state_c = self.lstm(x, initial_state=hidden)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)

        return x, [state_h, state_c], attention_weights

    def initialize_hidden_state(self):
        return [tf.zeros((self.batch_sz, self.dec_units), dtype=tf.float32)] * 2


In [49]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [57]:
# Define optimizer
optimizer = tf.optimizers.Adam(learning_rate=0.001)

# Define your loss function
def loss_function(real, pred):
    mask = 1 - tf.dtypes.cast(tf.math.equal(real, 0), tf.float32)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)


checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [1]:
# Define your target labels, which should match your actual target language vocabulary
target_labels = ['<start>', 'word1', 'word2']

# Create a dictionary to map target words to integer indices
targ_lang = {word: idx for idx, word in enumerate(target_labels)}
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()

    # Initialize the hidden state at the start of each epoch
    hidden = (tf.zeros((BATCH_SIZE, units)), tf.zeros((BATCH_SIZE, units)))
    # Initialize your target language dictionary
    targ_lang = {'<start>': 0, 'word1': 1, 'word2': 2}



    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0

        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)


            dec_hidden = enc_hidden

            dec_input = tf.expand_dims([targ_lang['<start>']] * BATCH_SIZE, 1)


            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

                loss += loss_function(targ[:, t], predictions)

                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)

        batch_loss = (loss / int(targ.shape[1]))
        total_loss += batch_loss

        variables = encoder.variables + decoder.variables

        gradients = tape.gradient(loss, variables)

        optimizer.apply_gradients(zip(gradients, variables))

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss.numpy()))

    # saving (checkpoint) the model every epoch
    checkpoint.save(file_prefix=checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))


NameError: ignored

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
def evaluate(inputs, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):


    attention_plot = np.zeros((max_length_targ, max_length_inp))
    sentence = ''
    for i in inputs[0]:
        if i == 0:
            break
        sentence = sentence + inp_lang.idx2word[i] + ' '
    sentence = sentence[:-1]

    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

def predict_random_val_sentence():
    actual_sent = ''
    k = np.random.randint(len(input_tensor_val))
    random_input = input_tensor_val[k]
    random_output = target_tensor_val[k]
    random_input = np.expand_dims(random_input,0)
    result, sentence, attention_plot = evaluate(random_input, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
    print('Input: {}'.format(sentence[8:-6]))
    print('Predicted translation: {}'.format(result[:-6]))
    for i in random_output:
        if i == 0:
            break
        actual_sent = actual_sent + targ_lang.idx2word[i] + ' '
    actual_sent = actual_sent[8:-7]
    print('Actual translation: {}'.format(actual_sent))
    attention_plot = attention_plot[:len(result.split(' '))-2, 1:len(sentence.split(' '))-1]
    sentence, result = sentence.split(' '), result.split(' ')
    sentence = sentence[1:-1]
    result = result[:-2]

    # use plotly to generate the heat map
    trace = go.Heatmap(z = attention_plot, x = sentence, y = result, colorscale='greens')
    data=[trace]
    iplot(data)



In [ ]:
predict_random_val_sentence()